<a href="https://colab.research.google.com/github/MudasirIrshad/Agentic_AI_Panaversity_Course/blob/main/PROJECT_DocumentBoot_RAG_FunctionCalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.2 MB/s eta 0:00:00


/usr/lib/python3.11/pathlib.py:69: RuntimeWarning: coroutine 'main' was never awaited
  for x in reversed(rel.split(sep)):


In [17]:
from google.colab import files
pdf = files.upload()

Saving AI_Trends.pdf to AI_Trends.pdf


In [2]:
!pip install -qU langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00


In [22]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/AI_Trends.pdf")
page = []
async for i in loader.alazy_load():
  page.append(i)


In [28]:
print(f"{page[20].metadata}\n")
print(page[20].page_content)

New Trends in the Use of Artificial Intelligence for the Industry 4.0
6
In contrast, technological projects depend on the result produced, mainly due to the 
diversity of tasks [24]. Furthermore, although complexity is usually expressed by 
the means of cost, duration, or people involved, these criteria do not correlate well 
with how they are managed [25].
In summary, complex projects consist of ambiguity and uncertainty, interde-
pendency, nonlinearity, unique local conditions, autonomy, emergent behaviors, 
and unfixed boundaries. According to these properties, projects can be classified 
as simple, complicated, chaotic, and complex [26]. On the other hand, complex 
projects are also influenced by significant external changes [27], from misaligned 
stakeholders’ view of success, in which current tools and decision processes are 
unsuitable for analyze it. To respond positively to this complexity, it is necessary to 
imply both organizations and practitioners [28].
It can be noticed 

In [29]:
%pip install --upgrade --quiet  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
